In [1]:
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Cropping2D, \
                                    Add, MaxPooling2D, Flatten, Dense, Activation
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.utils.vis_utils import plot_model

In [2]:
def residual_block(x, kernelsize, filters, downsample):
    lyr = Conv2D(filters, kernelsize, strides=1, padding='same')(x)
    lyr = ReLU()(lyr)
  #  lyr = BatchNormalization()(lyr)
    
    if downsample:
        lyr = Conv2D(filters, 1,strides=2 , padding='same')(lyr)
        #x = shortcut(x, lyr)
        shortcut = Conv2D(filters, kernel_size=(1, 1), strides=2, padding='same')(x)
       # shortcut = BatchNormalization()(shortcut)

    else:
        lyr = Conv2D(filters, kernelsize,  strides=1, padding='same')(lyr)
        shortcut = x
    out = Add()([shortcut,lyr])
    out = ReLU()(out)
  #  out = BatchNormalization()(out)
    return out

In [3]:
### ResNet Conv. Nural Network
# Define the input as a tensor with shape input_shape
filters = 16
width = 32
height = 32
ch = 3
X_input = Input(shape=(width, height, ch))
X = X_input
 
# Stage 1
X = Conv2D(filters, (7, 7))(X)#, kernel_initializer = glorot_uniform(seed=0))(X)
X = Activation('relu')(X)
X = MaxPooling2D((2, 2), strides=(2, 2))(X) 
##### Residual block 1 #####
X = residual_block(X, 3, filters, 1)
##### Residual block 2 #####
X = residual_block(X, 3, filters, 1)
##### Residual block 3 #####
#X = residual_block(X, 5, filters, 0)
##### Residual block 4 #####
X = residual_block(X, 3, filters, 1)
##### Residual block 5 #####
X = residual_block(X, 3, filters, 1)




X_output = Flatten()(X)
X_output = Dense(16, activation='relu')(X_output)
X_output = Dense(7, activation='relu')(X_output)
X_output = Dense(3, activation='softmax')(X_output)
    
model = Model(X_input, X_output,name = 'ResNet')

model.compile(optimizer='adam',   loss='categorical_crossentropy',  metrics=['accuracy'] )
             # loss='sparse_categorical_crossentropy'

model.summary()

Model: "ResNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 26, 26, 16)   2368        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 26, 26, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 13, 13, 16)   0           ['activation[0][0]']             
                                                                                             

In [4]:
plot_model(model,  show_shapes=True, show_layer_names=True)#to_file='model_plot.png',

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [5]:
# Save Model
model.save("FaceMask_model")

INFO:tensorflow:Assets written to: FaceMask_model\assets
